### Function to convert fits to json, given indices
Outputs to ./output_json

In [70]:
import json, math, numpy

def sph2cart(coord):
    return [
         coord[2] * math.sin(coord[1]) * math.cos(coord[0]), 
         coord[2] * math.sin(coord[1]) * math.sin(coord[0]),
         coord[2] * math.cos(coord[1])
    ]

def coord2datarow(coord):
    sph = sph2cart(coord[0:3])
    sph.append(abs(coord[5]))
    #ugly workaround for json to encode to single precision
    return [float(str(numpy.float32(x))) for x in sph]

In [74]:
from astropy.io import fits

def toJson(filename, modelname, indices,  typename):
    fl_fits = fits.open(filename)
    ot = fl_fits[0].header['OBSTIME']
    time = ot[0:4] + '-' + ot[5:7] + '-' + ot[8:10] + 'T'
    time += ot[11:13] + ot[14:17] + ot[18:21] + '.000'
    dict_to_write = {}
    for i in indices:
        fieldline = fl_fits[0].data[i]
        data = [coord2datarow(pt) for pt in fieldline if pt[0] > -900] 
        if (len(data) < 2):                                      
            continue
        adict = {
            str(i): {
                "time": time,
                 "trace": {
                     "columns": ["x","y","z","b-r"],
                     "data": data }}}
        dict_to_write.update(adict)
    
    fl_fits.close()
    outputname = './output_json/' + modelname + '_' + typename + '_' + time + '.json'
    with open(outputname, 'w') as outfile:
        json.dump(dict_to_write, outfile)

### Get all filenames
Directory name is fitsfiles. Deduce the model typ from the filename.

In [78]:
from os import walk

fitsfiles = [] 
for (dirpath, dirnames, filenames) in walk('./fitsfiles'):
    for file in filenames:    
        if(file.endswith('.fits')):
            if('pfss_intoout' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_IO'))
            elif('pfss_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_OI'))
            elif('scs_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'SCS_OI'))
    break

[('./fitsfiles/201905020800R000_trace_pfss_outtoin_tracing.fits', 'PFSS_OI'),
 ('./fitsfiles/201905020800R000_trace_scs_outtoin_tracing.fits', 'SCS_OI'),
 ('./fitsfiles/201905021200R000_trace_scs_outtoin_tracing.fits', 'SCS_OI'),
 ('./fitsfiles/201905021600R000_trace_pfss_outtoin_tracing.fits', 'PFSS_OI'),
 ('./fitsfiles/201905022000R000_trace_pfss_outtoin_tracing.fits', 'PFSS_OI'),
 ('./fitsfiles/201905021200R000_trace_pfss_outtoin_tracing.fits', 'PFSS_OI'),
 ('./fitsfiles/201905021600R000_trace_scs_outtoin_tracing.fits', 'SCS_OI'),
 ('./fitsfiles/201905020800R000_trace_pfss_intoout_tracing.fits', 'PFSS_IO'),
 ('./fitsfiles/201905022000R000_trace_scs_outtoin_tracing.fits', 'SCS_OI'),
 ('./fitsfiles/201905021200R000_trace_pfss_intoout_tracing.fits', 'PFSS_IO'),
 ('./fitsfiles/201905021600R000_trace_pfss_intoout_tracing.fits', 'PFSS_IO'),
 ('./fitsfiles/201905022000R000_trace_pfss_intoout_tracing.fits', 'PFSS_IO')]

### Create indices
Every n:th, boundary filtering, last 540 or whatever you like

In [66]:
def everynth(step):
    return (range(0,16200, step), 'step' + str(step))

indices = everynth(25)

### Write to json
This takes a few seconds

In [77]:
import time
start_time = time.time()
for fitsfile in fitsfiles:
    toJson(fitsfile[0], fitsfile[1], indices[0], indices[1])
    print('Finished converting {} after {} seconds: '.format(fitsfile[0],time.time()-start_time))
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))

Finished converting ./fitsfiles/201905020800R000_trace_pfss_outtoin_tracing.fits after 4.373383283615112 seconds: 
Finished converting ./fitsfiles/201905020800R000_trace_scs_outtoin_tracing.fits after 9.925315141677856 seconds: 
Finished converting ./fitsfiles/201905021200R000_trace_scs_outtoin_tracing.fits after 17.447620153427124 seconds: 
Finished converting ./fitsfiles/201905021600R000_trace_pfss_outtoin_tracing.fits after 23.61156725883484 seconds: 
Finished converting ./fitsfiles/201905022000R000_trace_pfss_outtoin_tracing.fits after 27.901528120040894 seconds: 
Finished converting ./fitsfiles/201905021200R000_trace_pfss_outtoin_tracing.fits after 33.988930225372314 seconds: 
Finished converting ./fitsfiles/201905021600R000_trace_scs_outtoin_tracing.fits after 41.508538246154785 seconds: 
Finished converting ./fitsfiles/201905020800R000_trace_pfss_intoout_tracing.fits after 42.88757610321045 seconds: 
Finished converting ./fitsfiles/201905021200R000_trace_pfss_intoout_tracing.fit